In [1]:
import os 
os.chdir('../../')
print("Current working directory is now: ", os.getcwd())

import pandas as pd 
import numpy as np
import csv
from sklearn.linear_model import LogisticRegression

import fl_utils.fl_interpretable_functions as interpret
import fl_utils.fl_riskslim as slim
import fl_utils.fl_stumps as stumps

Current working directory is now:  C:\Users\binha\Documents\Duke\Cynthia Research\interpretable-machine-learning\ky-fl-combined\fl-model


C:\Users\binha\Anaconda3\lib\site-packages\sklearn\externals\six.py:31: DeprecationWarning:

The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).



### EBM & CART

In [2]:
KY_data = pd.read_csv("~/Documents/Duke/Cynthia Research/data/ky-fl-data/KY-cleaned.csv").sort_values('person_id')
FL_data = pd.read_csv("~/Documents/Duke/Cynthia Research/data/ky-fl-data/FL-cleaned.csv").sort_values('person_id')

## split x and y 
KY_X = KY_data.loc[:,:'current_violence20']
KY_Y = KY_data['felony_six_month'].values
FL_X = FL_data.loc[:,:'current_violence20']
FL_Y = FL_data['felony_six_month'].values

In [3]:
#### CART
depth = [1,2,3,4,5]
impurity = [0.001, 0.002, 0.003, 0.004, 0.005]
cart_summary = interpret.CART(KY_X, KY_Y, FL_X, FL_Y, depth, impurity, seed=816)

### EBM
estimators = [40,60,80,100]
depth = [1,2,3]
learning_rate = [0.01]
holdout_split = [0.7, 0.9]
ebm_summary = interpret.EBM(KY_X, KY_Y, FL_X, FL_Y, learning_rate, depth, estimators, holdout_split, seed=816)

In [4]:
np.mean(cart_summary['KY_score']), np.mean(ebm_summary['KY_score'])

(0.5171696361067059, 0.7095323015260075)

### Lasso Stumps

In [5]:
KY_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/data/ky-fl-data/KY_stumps").sort_values('person_id')
FL_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/data/ky-fl-data/FL_stumps").sort_values('person_id')

## split x and y 
KY_X = KY_stumps.loc[:,:'current_violence201']
KY_Y = KY_stumps['felony_six_month'].values
FL_X = FL_stumps.loc[:,:'current_violence201']
FL_Y = FL_stumps['felony_six_month'].values

## columns 
cols = KY_X.columns[1:]

In [6]:
single_stump_model = stumps.stump_model(KY_X, 
                                        KY_Y, 
                                        FL_X, 
                                        FL_Y, 
                                        0.1, 
                                        cols, 
                                        816)
single_stump_model['features']

['age_at_current_charge30',
 'age_at_current_charge36',
 'age_at_current_charge40',
 'p_arrest1',
 'p_charges1',
 'p_charges2',
 'p_charges6',
 'p_violence2',
 'p_felony1',
 'p_felony3',
 'p_property2',
 'p_property3',
 'p_misdeassult1',
 'p_drug2',
 'p_drug3',
 'p_dui1',
 'p_stealing1',
 'p_fta_two_year2',
 'p_probation1',
 'p_incarceration1',
 'six_month1',
 'one_year1',
 'five_year1']

In [7]:
c_grid={'C': [0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1]}
stumps_summary = stumps.stump_cv(KY_X, KY_Y, FL_X, FL_Y, cols, c_grid, seed=816)

In [8]:
stumps_summary['best_params'], np.mean(stumps_summary['KY_score']), np.mean(stumps_summary['auc_diff'])

([{'C': 0.04}, {'C': 0.07}, {'C': 0.1}, {'C': 0.07}, {'C': 0.1}],
 0.6667161443260342,
 0.08433536706736174)

### RiskSLIM

In [9]:
single_stump_model = stumps.stump_model(KY_X, 
                                        KY_Y, 
                                        FL_X, 
                                        FL_Y,  
                                        0.03,  
                                        cols, 
                                        816)
selected_features = single_stump_model['features']
len(selected_features)

10

In [10]:
### Subset features
sub_FL_X = FL_stumps.loc[:, selected_features]
sub_KY_X = KY_stumps.loc[:, selected_features]
sub_FL_X.insert(0, '(Intercept)', 1)

In [11]:
riskslim_summary = slim.risk_cv(sub_KY_X, 
                                KY_Y, 
                                sub_FL_X, 
                                FL_Y, 
                                y_label = 'felony_six_month', 
                                max_coef = 5, 
                                max_coef_number = 5, 
                                max_runtime=1000,
                                max_offset=100,
                                c=1e-6, 
                                seed=816)

setting c0 = 0.0 to ensure that intercept is not penalized
12/27/19 @ 01:00 PM | 153 rows in lookup table
12/27/19 @ 01:00 PM | ------------------------------------------------------------
12/27/19 @ 01:00 PM | runnning initialization procedure
12/27/19 @ 01:00 PM | ------------------------------------------------------------
12/27/19 @ 01:00 PM | CPA produced 2 cuts
12/27/19 @ 01:00 PM | running naive rounding on 35 solutions
12/27/19 @ 01:00 PM | best objective value: 0.2936
12/27/19 @ 01:00 PM | rounding produced 4 integer solutions
12/27/19 @ 01:00 PM | best objective value is 0.3002
12/27/19 @ 01:00 PM | running sequential rounding on 35 solutions
12/27/19 @ 01:00 PM | best objective value: 0.2936
12/27/19 @ 01:00 PM | sequential rounding produced 2 integer solutions
12/27/19 @ 01:00 PM | best objective value: 0.3002
12/27/19 @ 01:00 PM | polishing 6 solutions
12/27/19 @ 01:00 PM | best objective value: 0.3002
12/27/19 @ 01:00 PM | polishing produced 5 integer solutions
12/27/19 @

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
12/27/19 @ 01:00 PM | adding 71 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.3002.
Tried aggregator 1 time.
Reduced MIP has 12 rows, 24 columns, and 43 nonzeros.
Reduced MIP has 10 binaries, 12 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.02 ticks)
Probing time = 0.00 sec. (0.00 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.02 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variable

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
12/27/19 @ 01:00 PM | adding 77 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.2960.
Tried aggregator 1 time.
Reduced MIP has 12 rows, 24 columns, and 43 nonzeros.
Reduced MIP has 10 binaries, 12 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.02 ticks)
Probing time = 0.00 sec. (0.00 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.02 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variable

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
12/27/19 @ 01:00 PM | adding 83 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.2928.
Tried aggregator 1 time.
Reduced MIP has 12 rows, 24 columns, and 43 nonzeros.
Reduced MIP has 10 binaries, 12 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.02 ticks)
Probing time = 0.00 sec. (0.00 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.02 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variable

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
12/27/19 @ 01:00 PM | adding 84 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.2933.
Tried aggregator 1 time.
Reduced MIP has 12 rows, 24 columns, and 43 nonzeros.
Reduced MIP has 10 binaries, 12 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.02 ticks)
Probing time = 0.00 sec. (0.00 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.02 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variable

...

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
12/27/19 @ 01:01 PM | adding 74 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.2960.
Tried aggregator 1 time.
Reduced MIP has 12 rows, 24 columns, and 43 nonzeros.
Reduced MIP has 10 binaries, 12 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.02 ticks)
Probing time = 0.00 sec. (0.00 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.02 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variable

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
12/27/19 @ 01:01 PM | adding 81 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.2870.
Tried aggregator 1 time.
Reduced MIP has 12 rows, 24 columns, and 43 nonzeros.
Reduced MIP has 10 binaries, 12 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.02 ticks)
Probing time = 0.00 sec. (0.00 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.02 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variable

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
12/27/19 @ 01:01 PM | adding 75 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.2883.
Tried aggregator 1 time.
Reduced MIP has 12 rows, 24 columns, and 43 nonzeros.
Reduced MIP has 10 binaries, 12 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.02 ticks)
Probing time = 0.00 sec. (0.00 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.02 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variable

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
12/27/19 @ 01:01 PM | adding 69 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.3014.
Tried aggregator 1 time.
Reduced MIP has 12 rows, 24 columns, and 43 nonzeros.
Reduced MIP has 10 binaries, 12 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (0.02 ticks)
Probing time = 0.00 sec. (0.00 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.02 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variable

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
12/27/19 @ 01:01 PM | adding 85 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.2935.
Tried aggregator 1 time.
Reduced MIP has 12 rows, 24 columns, and 43 nonzeros.
Reduced MIP has 10 binaries, 12 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.02 ticks)
Probing time = 0.00 sec. (0.00 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.02 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variable

In [12]:
np.mean(riskslim_summary['FL_validation']), np.mean(riskslim_summary['KY_score'])

(0.5711171264001176, 0.6241033098871197)

In [13]:
#### save results
summary_F6_fl_inter_model = {"cart": cart_summary,
                             "ebm": ebm_summary,
                             "stumps": stumps_summary,
                             "riskslim": riskslim_summary}

In [15]:
path = "./results/interpretable/six-month/"
results = [["Felony",             
            np.str(round(np.mean(cart_summary['KY_score']),3)) + " (" + np.str(round(np.std(cart_summary['KY_score']),3)) + ")", 
            np.str(round(np.mean(ebm_summary['KY_score']), 3)) + " (" + np.str(round(np.std(ebm_summary['KY_score']),3)) + ")", 
            np.str(round(np.mean(stumps_summary['KY_score']),3)) + " (" + np.str(round(np.std(stumps_summary['KY_score']),3)) + ")", 
            np.str(round(np.mean(riskslim_summary['KY_score']), 3)) + " (" + np.str(round(np.std(riskslim_summary['KY_score']),3)) + ")"]]
with open(path + 'six-month-fl-interpretable.csv', 'a') as writeFile:
    writer = csv.writer(writeFile)
    writer.writerows(results)